In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Set Random Seed

In [ ]:
np.random.seed(42)

# Read Data

In [ ]:
import warnings
warnings.filterwarnings("ignore")
import matplotlib.pyplot as plt

In [ ]:
train = pd.read_csv("../input/commonlitreadabilityprize/train.csv")

In [ ]:
train.shape

In [ ]:
text = train.excerpt
target = train.target

In [ ]:
train

In [ ]:
plt.plot(train.standard_error)

In [ ]:
train.excerpt[106]

# Use SBERT Embedding

In [ ]:
!ls ../input/sentence-transformers

In [ ]:
!mkdir -p /tmp/pip/cache/
!cp ../input/sentence-transformers/sacremoses-0.0.45-py3-none-any.whl /tmp/pip/cache
!cp ../input/sentence-transformers/sentence-transformers-1.1.0.xyz /tmp/pip/cache/sentence-transformers-1.1.0.tar
!cp ../input/sentence-transformers/sentencepiece-0.1.95-cp37-cp37m-manylinux2014_x86_64.whl /tmp/pip/cache
!cp ../input/sentence-transformers/tokenizers-0.10.2-cp37-cp37m-manylinux2010_x86_64.whl /tmp/pip/cache
!cp ../input/sentence-transformers/transformers-4.5.1-py3-none-any.whl /tmp/pip/cache

In [ ]:
!pip install --no-index --find-links /tmp/pip/cache/ sentence-transformers

In [ ]:
from sentence_transformers import SentenceTransformer

In [ ]:
model1 = SentenceTransformer('../input/paraphrasempnetbasev2',device='cuda')
#model2 = SentenceTransformer('../input/average-word-embeddings-komninos',device='cuda')
model3 = SentenceTransformer('../input/stsbdistilrobertabasev2',device='cuda')
model4 = SentenceTransformer('../input/stsbmpnetbasev2',device='cuda')
model5 = SentenceTransformer('../input/stsbrobertabasev2',device='cuda')

In [ ]:
model1.max_seq_length = 550
#model2.max_seq_length = 550
model3.max_seq_length = 550
model4.max_seq_length = 550
model5.max_seq_length = 550

In [ ]:
document_embeddings1 = pd.DataFrame(model1.encode(text))
#document_embeddings2 = pd.DataFrame(model2.encode(text))
document_embeddings3 = pd.DataFrame(model3.encode(text))
document_embeddings4 = pd.DataFrame(model4.encode(text))
document_embeddings5 = pd.DataFrame(model5.encode(text))

In [ ]:
print(document_embeddings1.shape)
#print(document_embeddings2.shape)
print(document_embeddings3.shape)
print(document_embeddings4.shape)
print(document_embeddings5.shape)

# Readability Metrics

In [ ]:
import nltk
import math
import re
import spacy
nlp = spacy.load('en')

In [ ]:
def syllable_count(word):
    #referred from stackoverflow.com/questions/14541303/count-the-number-of-syllables-in-a-word
    word = word.lower()
    count = 0
    vowels = "aeiouy"
    if word[0] in vowels:
        count += 1
    for index in range(1, len(word)):
        if word[index] in vowels and word[index - 1] not in vowels:
            count += 1
    if word.endswith("e"):
        count -= 1
    if word.endswith('le'):
        count += 1
    if count == 0:
        count += 1
    return count

In [ ]:
#refer to https://www.kaggle.com/accountstatus/readability-index-in-python/comments#Flesch-Formula
# Function to get the adjusted score using the new dale chall formula
dale_chall_words="a|able|aboard|about|above|absent|accept|accident|account|ache|aching|acorn|acre|across|act|acts|add|address|admire|adventure|afar|afraid|after|afternoon|afterward|afterwards|again|against|age|aged|ago|agree|ah|ahead|aid|aim|air|airfield|airplane|airport|airship|airy|alarm|alike|alive|all|alley|alligator|allow|almost|alone|along|aloud|already|also|always|am|America|American|among|amount|an|and|angel|anger|angry|animal|another|answer|ant|any|anybody|anyhow|anyone|anything|anyway|anywhere|apart|apartment|ape|apiece|appear|apple|April|apron|are|aren't|arise|arithmetic|arm|armful|army|arose|around|arrange|arrive|arrived|arrow|art|artist|as|ash|ashes|aside|ask|asleep|at|ate|attack|attend|attention|August|aunt|author|auto|automobile|autumn|avenue|awake|awaken|away|awful|awfully|awhile|ax|axe|baa|babe|babies|back|background|backward|backwards|bacon|bad|badge|badly|bag|bake|baker|bakery|baking|ball|balloon|banana|band|bandage|bang|banjo|bank|banker|bar|barber|bare|barefoot|barely|bark|barn|barrel|base|baseball|basement|basket|bat|batch|bath|bathe|bathing|bathroom|bathtub|battle|battleship|bay|be|beach|bead|beam|bean|bear|beard|beast|beat|beating|beautiful|beautify|beauty|became|because|become|becoming|bed|bedbug|bedroom|bedspread|bedtime|bee|beech|beef|beefsteak|beehive|been|beer|beet|before|beg|began|beggar|begged|begin|beginning|begun|behave|behind|being|believe|bell|belong|below|belt|bench|bend|beneath|bent|berries|berry|beside|besides|best|bet|better|between|bib|bible|bicycle|bid|big|bigger|bill|billboard|bin|bind|bird|birth|birthday|biscuit|bit|bite|biting|bitter|black|blackberry|blackbird|blackboard|blackness|blacksmith|blame|blank|blanket|blast|blaze|bleed|bless|blessing|blew|blind|blindfold|blinds|block|blood|bloom|blossom|blot|blow|blue|blueberry|bluebird|blush|board|boast|boat|bob|bobwhite|bodies|body|boil|boiler|bold|bone|bonnet|boo|book|bookcase|bookkeeper|boom|boot|born|borrow|boss|both|bother|bottle|bottom|bought|bounce|bow|bowl|bow-wow|box|boxcar|boxer|boxes|boy|boyhood|bracelet|brain|brake|bran|branch|brass|brave|bread|break|breakfast|breast|breath|breathe|breeze|brick|bride|bridge|bright|brightness|bring|broad|broadcast|broke|broken|brook|broom|brother|brought|brown|brush|bubble|bucket|buckle|bud|buffalo|bug|buggy|build|building|built|bulb|bull|bullet|bum|bumblebee|bump|bun|bunch|bundle|bunny|burn|burst|bury|bus|bush|bushel|business|busy|but|butcher|butt|butter|buttercup|butterfly|buttermilk|butterscotch|button|buttonhole|buy|buzz|by|bye|cab|cabbage|cabin|cabinet|cackle|cage|cake|calendar|calf|call|caller|calling|came|camel|camp|campfire|can|canal|canary|candle|candlestick|candy|cane|cannon|cannot|canoe|can't|canyon|cap|cape|capital|captain|car|card|cardboard|care|careful|careless|carelessness|carload|carpenter|carpet|carriage|carrot|carry|cart|carve|case|cash|cashier|castle|cat|catbird|catch|catcher|caterpillar|catfish|catsup|cattle|caught|cause|cave|ceiling|cell|cellar|cent|center|cereal|certain|certainly|chain|chair|chalk|champion|chance|change|chap|charge|charm|chart|chase|chatter|cheap|cheat|check|checkers|cheek|cheer|cheese|cherry|chest|chew|chick|chicken|chief|child|childhood|children|chill|chilly|chimney|chin|china|chip|chipmunk|chocolate|choice|choose|chop|chorus|chose|chosen|christen|Christmas|church|churn|cigarette|circle|circus|citizen|city|clang|clap|class|classmate|classroom|claw|clay|clean|cleaner|clear|clerk|clever|click|cliff|climb|clip|cloak|clock|close|closet|cloth|clothes|clothing|cloud|cloudy|clover|clown|club|cluck|clump|coach|coal|coast|coat|cob|cobbler|cocoa|coconut|cocoon|cod|codfish|coffee|coffeepot|coin|cold|collar|college|color|colored|colt|column|comb|come|comfort|comic|coming|company|compare|conductor|cone|connect|coo|cook|cooked|cooking|cookie|cookies|cool|cooler|coop|copper|copy|cord|cork|corn|corner|correct|cost|cot|cottage|cotton|couch|cough|could|couldn't|count|counter|country|county|course|court|cousin|cover|cow|coward|cowardly|cowboy|cozy|crab|crack|cracker|cradle|cramps|cranberry|crank|cranky|crash|crawl|crazy|cream|creamy|creek|creep|crept|cried|croak|crook|crooked|crop|cross|crossing|cross-eyed|crow|crowd|crowded|crown|cruel|crumb|crumble|crush|crust|cry|cries|cub|cuff|cup|cuff|cup|cupboard|cupful|cure|curl|curly|curtain|curve|cushion|custard|customer|cut|cute|cutting|dab|dad|daddy|daily|dairy|daisy|dam|damage|dame|damp|dance|dancer|dancing|dandy|danger|dangerous|dare|dark|darkness|darling|darn|dart|dash|date|daughter|dawn|day|daybreak|daytime|dead|deaf|deal|dear|death|December|decide|deck|deed|deep|deer|defeat|defend|defense|delight|den|dentist|depend|deposit|describe|desert|deserve|desire|desk|destroy|devil|dew|diamond|did|didn't|die|died|dies|difference|different|dig|dim|dime|dine|ding-dong|dinner|dip|direct|direction|dirt|dirty|discover|dish|dislike|dismiss|ditch|dive|diver|divide|do|dock|doctor|does|doesn't|dog|doll|dollar|dolly|done|donkey|don't|door|doorbell|doorknob|doorstep|dope|dot|double|dough|dove|down|downstairs|downtown|dozen|drag|drain|drank|draw|drawer|draw|drawing|dream|dress|dresser|dressmaker|drew|dried|drift|drill|drink|drip|drive|driven|driver|drop|drove|drown|drowsy|drub|drum|drunk|dry|duck|due|dug|dull|dumb|dump|during|dust|dusty|duty|dwarf|dwell|dwelt|dying|each|eager|eagle|ear|early|earn|earth|east|eastern|easy|eat|eaten|edge|egg|eh|eight|eighteen|eighth|eighty|either|elbow|elder|eldest|electric|electricity|elephant|eleven|elf|elm|else|elsewhere|empty|end|ending|enemy|engine|engineer|English|enjoy|enough|enter|envelope|equal|erase|eraser|errand|escape|eve|even|evening|ever|every|everybody|everyday|everyone|everything|everywhere|evil|exact|except|exchange|excited|exciting|excuse|exit|expect|explain|extra|eye|eyebrow|fable|face|facing|fact|factory|fail|faint|fair|fairy|faith|fake|fall|false|family|fan|fancy|far|faraway|fare|farmer|farm|farming|far-off|farther|fashion|fast|fasten|fat|father|fault|favor|favorite|fear|feast|feather|February|fed|feed|feel|feet|fell|fellow|felt|fence|fever|few|fib|fiddle|field|fife|fifteen|fifth|fifty|fig|fight|figure|file|fill|film|finally|find|fine|finger|finish|fire|firearm|firecracker|fireplace|fireworks|firing|first|fish|fisherman|fist|fit|fits|five|fix|flag|flake|flame|flap|flash|flashlight|flat|flea|flesh|flew|flies|flight|flip|flip-flop|float|flock|flood|floor|flop|flour|flow|flower|flowery|flutter|fly|foam|fog|foggy|fold|folks|follow|following|fond|food|fool|foolish|foot|football|footprint|for|forehead|forest|forget|forgive|forgot|forgotten|fork|form|fort|forth|fortune|forty|forward|fought|found|fountain|four|fourteen|fourth|fox|frame|free|freedom|freeze|freight|French|fresh|fret|Friday|fried|friend|friendly|friendship|frighten|frog|from|front|frost|frown|froze|fruit|fry|fudge|fuel|full|fully|fun|funny|fur|furniture|further|fuzzy|gain|gallon|gallop|game|gang|garage|garbage|garden|gas|gasoline|gate|gather|gave|gay|gear|geese|general|gentle|gentleman|gentlemen|geography|get|getting|giant|gift|gingerbread|girl|give|given|giving|glad|gladly|glance|glass|glasses|gleam|glide|glory|glove|glow|glue|go|going|goes|goal|goat|gobble|God|god|godmother|gold|golden|goldfish|golf|gone|good|goods|goodbye|good-by|goodbye|good-bye|good-looking|goodness|goody|goose|gooseberry|got|govern|government|gown|grab|gracious|grade|grain|grand|grandchild|grandchildren|granddaughter|grandfather|grandma|grandmother|grandpa|grandson|grandstand|grape|grapes|grapefruit|grass|grasshopper|grateful|grave|gravel|graveyard|gravy|gray|graze|grease|great|green|greet|grew|grind|groan|grocery|ground|group|grove|grow|guard|guess|guest|guide|gulf|gum|gun|gunpowder|guy|ha|habit|had|hadn't|hail|hair|haircut|hairpin|half|hall|halt|ham|hammer|hand|handful|handkerchief|handle|handwriting|hang|happen|happily|happiness|happy|harbor|hard|hardly|hardship|hardware|hare|hark|harm|harness|harp|harvest|has|hasn't|haste|hasten|hasty|hat|hatch|hatchet|hate|haul|have|haven't|having|hawk|hay|hayfield|haystack|he|head|headache|heal|health|healthy|heap|hear|hearing|heard|heart|heat|heater|heaven|heavy|he'd|heel|height|held|hell|he'll|hello|helmet|help|helper|helpful|hem|hen|henhouse|her|hers|herd|here|here's|hero|herself|he's|hey|hickory|hid|hidden|hide|high|highway|hill|hillside|hilltop|hilly|him|himself|hind|hint|hip|hire|his|hiss|history|hit|hitch|hive|ho|hoe|hog|hold|holder|hole|holiday|hollow|holy|home|homely|homesick|honest|honey|honeybee|honeymoon|honk|honor|hood|hoof|hook|hoop|hop|hope|hopeful|hopeless|horn|horse|horseback|horseshoe|hose|hospital|host|hot|hotel|hound|hour|house|housetop|housewife|housework|how|however|howl|hug|huge|hum|humble|hump|hundred|hung|hunger|hungry|hunk|hunt|hunter|hurrah|hurried|hurry|hurt|husband|hush|hut|hymn|I|ice|icy|I'd|idea|ideal|if|ill|I'll|I'm|important|impossible|improve|in|inch|inches|income|indeed|Indian|indoors|ink|inn|insect|inside|instant|instead|insult|intend|interested|interesting|into|invite|iron|is|island|isn't|it|its|it's|itself|I've|ivory|ivy|jacket|jacks|jail|jam|January|jar|jaw|jay|jelly|jellyfish|jerk|jig|job|jockey|join|joke|joking|jolly|journey|joy|joyful|joyous|judge|jug|juice|juicy|July|jump|June|junior|junk|just|keen|keep|kept|kettle|key|kick|kid|kill|killed|kind|kindly|kindness|king|kingdom|kiss|kitchen|kite|kitten|kitty|knee|kneel|knew|knife|knit|knives|knob|knock|knot|know|known|lace|lad|ladder|ladies|lady|laid|lake|lamb|lame|lamp|land|lane|language|lantern|lap|lard|large|lash|lass|last|late|laugh|laundry|law|lawn|lawyer|lay|lazy|lead|leader|leaf|leak|lean|leap|learn|learned|least|leather|leave|leaving|led|left|leg|lemon|lemonade|lend|length|less|lesson|let|let's|letter|letting|lettuce|level|liberty|library|lice|lick|lid|lie|life|lift|light|lightness|lightning|like|likely|liking|lily|limb|lime|limp|line|linen|lion|lip|list|listen|lit|little|live|lives|lively|liver|living|lizard|load|loaf|loan|loaves|lock|locomotive|log|lone|lonely|lonesome|long|look|lookout|loop|loose|lord|lose|loser|loss|lost|lot|loud|love|lovely|lover|low|luck|lucky|lumber|lump|lunch|lying|ma|machine|machinery|mad|made|magazine|magic|maid|mail|mailbox|mailman|major|make|making|male|mama|mamma|man|manager|mane|manger|many|map|maple|marble|march|March|mare|mark|market|marriage|married|marry|mask|mast|master|mat|match|matter|mattress|may|May|maybe|mayor|maypole|me|meadow|meal|mean|means|meant|measure|meat|medicine|meet|meeting|melt|member|men|mend|meow|merry|mess|message|met|metal|mew|mice|middle|midnight|might|mighty|mile|milk|milkman|mill|miler|million|mind|mine|miner|mint|minute|mirror|mischief|miss|Miss|misspell|mistake|misty|mitt|mitten|mix|moment|Monday|money|monkey|month|moo|moon|moonlight|moose|mop|more|morning|morrow|moss|most|mostly|mother|motor|mount|mountain|mouse|mouth|move|movie|movies|moving|mow|Mr.|Mrs.|much|mud|muddy|mug|mule|multiply|murder|music|must|my|myself|nail|name|nap|napkin|narrow|nasty|naughty|navy|near|nearby|nearly|neat|neck|necktie|need|needle|needn't|Negro|neighbor|neighborhood|neither|nerve|nest|net|never|nevermore|new|news|newspaper|next|nibble|nice|nickel|night|nightgown|nine|nineteen|ninety|no|nobody|nod|noise|noisy|none|noon|nor|north|northern|nose|not|note|nothing|notice|November|now|nowhere|number|nurse|nut|oak|oar|oatmeal|oats|obey|ocean|o'clock|October|odd|of|off|offer|office|officer|often|oh|oil|old|old-fashioned|on|once|one|onion|only|onward|open|or|orange|orchard|order|ore|organ|other|otherwise|ouch|ought|our|ours|ourselves|out|outdoors|outfit|outlaw|outline|outside|outward|oven|over|overalls|overcoat|overeat|overhead|overhear|overnight|overturn|owe|owing|owl|own|owner|ox|pa|pace|pack|package|pad|page|paid|pail|pain|painful|paint|painter|painting|pair|pal|palace|pale|pan|pancake|pane|pansy|pants|papa|paper|parade|pardon|parent|park|part|partly|partner|party|pass|passenger|past|paste|pasture|pat|patch|path|patter|pave|pavement|paw|pay|payment|pea|peas|peace|peaceful|peach|peaches|peak|peanut|pear|pearl|peck|peek|peel|peep|peg|pen|pencil|penny|people|pepper|peppermint|perfume|perhaps|person|pet|phone|piano|pick|pickle|picnic|picture|pie|piece|pig|pigeon|piggy|pile|pill|pillow|pin|pine|pineapple|pink|pint|pipe|pistol|pit|pitch|pitcher|pity|place|plain|plan|plane|plant|plate|platform|platter|play|player|playground|playhouse|playmate|plaything|pleasant|please|pleasure|plenty|plow|plug|plum|pocket|pocketbook|poem|point|poison|poke|pole|police|policeman|polish|polite|pond|ponies|pony|pool|poor|pop|popcorn|popped|porch|pork|possible|post|postage|postman|pot|potato|potatoes|pound|pour|powder|power|powerful|praise|pray|prayer|prepare|present|pretty|price|prick|prince|princess|print|prison|prize|promise|proper|protect|proud|prove|prune|public|puddle|puff|pull|pump|pumpkin|punch|punish|pup|pupil|puppy|pure|purple|purse|push|puss|pussy|pussycat|put|putting|puzzle|quack|quart|quarter|queen|queer|question|quick|quickly|quiet|quilt|quit|quite|rabbit|race|rack|radio|radish|rag|rail|railroad|railway|rain|rainy|rainbow|raise|raisin|rake|ram|ran|ranch|rang|rap|rapidly|rat|rate|rather|rattle|raw|ray|reach|read|reader|reading|ready|real|really|reap|rear|reason|rebuild|receive|recess|record|red|redbird|redbreast|refuse|reindeer|rejoice|remain|remember|remind|remove|rent|repair|repay|repeat|report|rest|return|review|reward|rib|ribbon|rice|rich|rid|riddle|ride|rider|riding|right|rim|ring|rip|ripe|rise|rising|river|road|roadside|roar|roast|rob|robber|robe|robin|rock|rocky|rocket|rode|roll|roller|roof|room|rooster|root|rope|rose|rosebud|rot|rotten|rough|round|route|row|rowboat|royal|rub|rubbed|rubber|rubbish|rug|rule|ruler|rumble|run|rung|runner|running|rush|rust|rusty|rye|sack|sad|saddle|sadness|safe|safety|said|sail|sailboat|sailor|saint|salad|sale|salt|same|sand|sandy|sandwich|sang|sank|sap|sash|sat|satin|satisfactory|Saturday|sausage|savage|save|savings|saw|say|scab|scales|scare|scarf|school|schoolboy|schoolhouse|schoolmaster|schoolroom|scorch|score|scrap|scrape|scratch|scream|screen|screw|scrub|sea|seal|seam|search|season|seat|second|secret|see|seeing|seed|seek|seem|seen|seesaw|select|self|selfish|sell|send|sense|sent|sentence|separate|September|servant|serve|service|set|setting|settle|settlement|seven|seventeen|seventh|seventy|several|sew|shade|shadow|shady|shake|shaker|shaking|shall|shame|shan't|shape|share|sharp|shave|she|she'd|she'll|she's|shear|shears|shed|sheep|sheet|shelf|shell|shepherd|shine|shining|shiny|ship|shirt|shock|shoe|shoemaker|shone|shook|shoot|shop|shopping|shore|short|shot|should|shoulder|shouldn't|shout|shovel|show|shower|shut|shy|sick|sickness|side|sidewalk|sideways|sigh|sight|sign|silence|silent|silk|sill|silly|silver|simple|sin|since|sing|singer|single|sink|sip|sir|sis|sissy|sister|sit|sitting|six|sixteen|sixth|sixty|size|skate|skater|ski|skin|skip|skirt|sky|slam|slap|slate|slave|sled|sleep|sleepy|sleeve|sleigh|slept|slice|slid|slide|sling|slip|slipped|slipper|slippery|slit|slow|slowly|sly|smack|small|smart|smell|smile|smoke|smooth|snail|snake|snap|snapping|sneeze|snow|snowy|snowball|snowflake|snuff|snug|so|soak|soap|sob|socks|sod|soda|sofa|soft|soil|sold|soldier|sole|some|somebody|somehow|someone|something|sometime|sometimes|somewhere|son|song|soon|sore|sorrow|sorry|sort|soul|sound|soup|sour|south|southern|space|spade|spank|sparrow|speak|speaker|spear|speech|speed|spell|spelling|spend|spent|spider|spike|spill|spin|spinach|spirit|spit|splash|spoil|spoke|spook|spoon|sport|spot|spread|spring|springtime|sprinkle|square|squash|squeak|squeeze|squirrel|stable|stack|stage|stair|stall|stamp|stand|star|stare|start|starve|state|station|stay|steak|steal|steam|steamboat|steamer|steel|steep|steeple|steer|stem|step|stepping|stick|sticky|stiff|still|stillness|sting|stir|stitch|stock|stocking|stole|stone|stood|stool|stoop|stop|stopped|stopping|store|stork|stories|storm|stormy|story|stove|straight|strange|stranger|strap|straw|strawberry|stream|street|stretch|string|strip|stripes|strong|stuck|study|stuff|stump|stung|subject|such|suck|sudden|suffer|sugar|suit|sum|summer|sun|Sunday|sunflower|sung|sunk|sunlight|sunny|sunrise|sunset|sunshine|supper|suppose|sure|surely|surface|surprise|swallow|swam|swamp|swan|swat|swear|sweat|sweater|sweep|sweet|sweetness|sweetheart|swell|swept|swift|swim|swimming|swing|switch|sword|swore|table|tablecloth|tablespoon|tablet|tack|tag|tail|tailor|take|taken|taking|tale|talk|talker|tall|tame|tan|tank|tap|tape|tar|tardy|task|taste|taught|tax|tea|teach|teacher|team|tear|tease|teaspoon|teeth|telephone|tell|temper|ten|tennis|tent|term|terrible|test|than|thank|thanks|thankful|Thanksgiving|that|that's|the|theater|thee|their|them|then|there|these|they|they'd|they'll|they're|they've|thick|thief|thimble|thin|thing|think|third|thirsty|thirteen|thirty|this|thorn|those|though|thought|thousand|thread|three|threw|throat|throne|through|throw|thrown|thumb|thunder|Thursday|thy|tick|ticket|tickle|tie|tiger|tight|till|time|tin|tinkle|tiny|tip|tiptoe|tire|tired|title|to|toad|toadstool|toast|tobacco|today|toe|together|toilet|told|tomato|tomorrow|ton|tone|tongue|tonight|too|took|tool|toot|tooth|toothbrush|toothpick|top|tore|torn|toss|touch|tow|toward|towards|towel|tower|town|toy|trace|track|trade|train|tramp|trap|tray|treasure|treat|tree|trick|tricycle|tried|trim|trip|trolley|trouble|truck|true|truly|trunk|trust|truth|try|tub|Tuesday|tug|tulip|tumble|tune|tunnel|turkey|turn|turtle|twelve|twenty|twice|twig|twin|two|ugly|umbrella|uncle|under|understand|underwear|undress|unfair|unfinished|unfold|unfriendly|unhappy|unhurt|uniform|United States|unkind|unknown|unless|unpleasant|until|unwilling|up|upon|upper|upset|upside|upstairs|uptown|upward|us|use|used|useful|valentine|valley|valuable|value|vase|vegetable|velvet|very|vessel|victory|view|village|vine|violet|visit|visitor|voice|vote|wag|wagon|waist|wait|wake|waken|walk|wall|walnut|want|war|warm|warn|was|wash|washer|washtub|wasn't|waste|watch|watchman|water|watermelon|waterproof|wave|wax|way|wayside|we|weak|weakness|weaken|wealth|weapon|wear|weary|weather|weave|web|we'd|wedding|Wednesday|wee|weed|week|we'll|weep|weigh|welcome|well|went|were|we're|west|western|wet|we've|whale|what|what's|wheat|wheel|when|whenever|where|which|while|whip|whipped|whirl|whisky|whiskey|whisper|whistle|white|who|who'd|whole|who'll|whom|who's|whose|why|wicked|wide|wife|wiggle|wild|wildcat|will|willing|willow|win|wind|windy|windmill|window|wine|wing|wink|winner|winter|wipe|wire|wise|wish|wit|witch|with|without|woke|wolf|woman|women|won|wonder|wonderful|won't|wood|wooden|woodpecker|woods|wool|woolen|word|wore|work|worker|workman|world|worm|worn|worry|worse|worst|worth|would|wouldn't|wound|wove|wrap|wrapped|wreck|wren|wring|write|writing|written|wrong|wrote|wrung|yard|yarn|year|yell|yellow|yes|yesterday|yet|yolk|yonder|you|you'd|you'll|young|youngster|your|yours|you're|yourself|yourselves|youth|you've"
dale_chall_list=dale_chall_words.split('|')

# Making a function for dale chall score
def dale_chall(text) :
    
    # Removing all the unwanted characters
    text2=re.sub(r'[^\w\s]','', text) 
    
    difficult_words=0
    words=0
    for i in text2.split(' ') :
        words+=1
        if i.lower() not in dale_chall_words :
            difficult_words+=1
            
    # finding the percentage of the difficult words in the text
    pdw=difficult_words*100/words
    
    # Finding the sentences in the text using spacy
    doc=nlp(text)
    sents=doc.sents
    
    # Finding the avg word length of the sentences
    avg=[]
    for i in sents :
        words=0
        words += len([token for token in i])
        avg.append(words)
    
    asl=sum(avg)/len(avg)
    
    # Calculating the raw score
    raw_score=0.1579*pdw + 0.0496*asl
    
    # Conditions for the difficult words is that if the percentage of the difficult words is more than 5 percent
    # 3.6365 is added to the raw score else it remains the same
    if pdw>=5 :
        raw_score+=3.6365
        
    # Returning the score
    return raw_score

In [ ]:
document_embeddings1["S1"] = 0
document_embeddings1["S2"] = 0
document_embeddings1["S3"] = 0
document_embeddings1["S4"] = 0
for j in range(len(text)):
    s1 = 0
    s2 = 0
    s3 = 0
    s4 = 0
    for i in nltk.tokenize.word_tokenize(text[j]):
        syllable = syllable_count(i)
        if syllable == 1:
            s1 += 1
        if syllable == 2:
            s2 += 1
        if syllable == 3:
            s3 += 1
        if syllable > 3:
            s4 += 1
    document_embeddings1["S1"][j] = s1
    document_embeddings1["S2"][j] = s2
    document_embeddings1["S3"][j] = s3
    document_embeddings1["S4"][j] = s4

In [ ]:
document_embeddings1["sent_count"] = text.apply(nltk.tokenize.sent_tokenize).apply(len)
document_embeddings1["word_count"] = text.apply(nltk.tokenize.word_tokenize).apply(len)

In [ ]:
document_embeddings1['dale_chall_score']=text.apply(dale_chall)

In [ ]:
document_embeddings = document_embeddings1.copy()
#document_embeddings=document_embeddings.merge(document_embeddings2,left_index=True,right_index=True,suffixes=('_x', '_y'))
document_embeddings=document_embeddings.merge(document_embeddings3,left_index=True,right_index=True,suffixes=('_z', '_v'))
document_embeddings=document_embeddings.merge(document_embeddings4,left_index=True,right_index=True,suffixes=('_a', 's'))
document_embeddings=document_embeddings.merge(document_embeddings5,left_index=True,right_index=True,suffixes=('_d', 'f'))

In [ ]:
document_embeddings

# Deep Learning Model

In [ ]:
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import callbacks
from tensorflow.keras import optimizers
from sklearn.model_selection import train_test_split

In [ ]:
from sklearn.metrics import mean_squared_error

In [ ]:
test = pd.read_csv("../input/commonlitreadabilityprize/test.csv")

In [ ]:
txt_test = test.excerpt

In [ ]:
test_embeddings1 = pd.DataFrame(model1.encode(txt_test))
#test_embeddings2 = pd.DataFrame(model2.encode(txt_test))
test_embeddings3 = pd.DataFrame(model3.encode(txt_test))
test_embeddings4 = pd.DataFrame(model4.encode(txt_test))
test_embeddings5 = pd.DataFrame(model5.encode(txt_test))

In [ ]:
test_embeddings1["S1"] = 0
test_embeddings1["S2"] = 0
test_embeddings1["S3"] = 0
test_embeddings1["S4"] = 0
for j in range(len(txt_test)):
    s1 = 0
    s2 = 0
    s3 = 0
    s4 = 0
    for i in nltk.tokenize.word_tokenize(txt_test[j]):
        syllable = syllable_count(i)
        if syllable == 1:
            s1 += 1
        if syllable == 2:
            s2 += 1
        if syllable == 3:
            s3 += 1
        if syllable > 3:
            s4 += 1
    test_embeddings1["S1"][j] = s1
    test_embeddings1["S2"][j] = s2
    test_embeddings1["S3"][j] = s3
    test_embeddings1["S4"][j] = s4

In [ ]:
test_embeddings1["sent_count"] = txt_test.apply(nltk.tokenize.sent_tokenize).apply(len)
test_embeddings1["word_count"] = txt_test.apply(nltk.tokenize.word_tokenize).apply(len)

In [ ]:
test_embeddings1['dale_chall_score']=txt_test.apply(dale_chall)

In [ ]:
test_embeddings = test_embeddings1.copy()
#test_embeddings=test_embeddings.merge(test_embeddings2,left_index=True,right_index=True,suffixes=('_x', '_y'))
test_embeddings=test_embeddings.merge(test_embeddings3,left_index=True,right_index=True,suffixes=('_z', '_v'))
test_embeddings=test_embeddings.merge(test_embeddings4,left_index=True,right_index=True,suffixes=('_a', 's'))
test_embeddings=test_embeddings.merge(test_embeddings5,left_index=True,right_index=True,suffixes=('_d', 'f'))

In [ ]:
datalist = [document_embeddings]
testlist = [test_embeddings]
pred1,pred2,pred3,pred4,pred5 =[0]*7,[0]*7,[0]*7,[0]*7,[0]*7
predlist = [pred1]

In [ ]:
def pred_data(datalist,testlist,pred):
    for i in range(len(datalist)):
        X_train, X_test, y_train, y_test = train_test_split(datalist[i], target, test_size = 0.3,random_state=42)
        
        if X_train.shape[1] >= 768:
            dl_model = keras.Sequential([
            # the hidden ReLU layers
            layers.Dense(units=2048,kernel_initializer='glorot_uniform',activation='relu', kernel_regularizer='l2', bias_regularizer="l1",input_shape=[X_train.shape[1]]),
            layers.Dense(units=1024, kernel_initializer='glorot_uniform',activation='relu', kernel_regularizer='l2', bias_regularizer="l1"),
            layers.Dense(units=512, kernel_initializer='glorot_uniform',activation='relu', kernel_regularizer='l2', bias_regularizer="l1"),
            layers.Dense(units=256, kernel_initializer='glorot_uniform',activation='relu', kernel_regularizer='l2', bias_regularizer="l1"),
            layers.Dense(units=128, kernel_initializer='glorot_uniform',activation='relu', kernel_regularizer='l2', bias_regularizer="l1"),
            #layers.Dense(units=64, kernel_initializer='glorot_uniform',activation='relu'),
            # the linear output layer 
            layers.Dense(units=1, kernel_initializer='glorot_uniform', kernel_regularizer='l2', bias_regularizer="l1"),
            ])
        else:
            dl_model = keras.Sequential([
            # the hidden ReLU layers
            layers.Dense(units=X_train.shape[1], activation='relu', input_shape=[X_train.shape[1]]),
            layers.Dense(units=256, activation='relu'),
            layers.Dense(units=128, activation='relu'),
            layers.Dense(units=64, activation='relu'),
            # the linear output layer 
            layers.Dense(units=1),
            ]) 

        # Define an early stopping callback
        early_stopping = callbacks.EarlyStopping(
            patience=30,
            min_delta=0.0001,
            restore_best_weights=True,
            )

        dl_model.compile(
            optimizer=optimizers.Adam(2e-5,beta_1=0.9, beta_2=0.98, epsilon=1e-06),
            loss='mse',
            metrics = [keras.metrics.RootMeanSquaredError()]
            )

        dl_model.summary()

        
        history = dl_model.fit(
            X_train, y_train,
            validation_data=(X_test, y_test),
            batch_size=32,
            epochs=200,
            callbacks=[early_stopping]
            )
        
        pred[i] = dl_model.predict(testlist[i])
        prediction = sum(pred)/len(pred)
    return prediction, history

In [ ]:
def plot_loss(history):
  plt.plot(history.history['loss'], label='loss')
  plt.plot(history.history['val_loss'], label='val_loss')
  plt.ylim([0, 100])
  plt.xlabel('Epoch')
  plt.ylabel('Error [MPG]')
  plt.legend()
  plt.grid(True)

# Submission

In [ ]:
test_preds, history = pred_data(datalist,testlist,predlist)

In [ ]:
hist = pd.DataFrame(history.history)
hist['epoch'] = history.epoch
hist[hist.val_root_mean_squared_error==min(hist.val_root_mean_squared_error)]

In [ ]:
plot_loss(history)

In [ ]:
submission = test[["id"]].copy()
submission["target"] = test_preds
submission.to_csv('submission.csv', index = False)
submission